In [5]:
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = "retina"

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))


/tmp/ipykernel_3190/945712042.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [4]:
import pandas as pd
import os
import argparse
from typing import Any, Callable, Sequence
from pathlib import Path
from PIL import Image
import matplotlib.image as mpimg

import jax
import jax.random as jr
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax.flatten_util import ravel_pytree
import numpy as np
import matplotlib.pyplot as plt
from flax import linen as nn


from bong.util import run_rebayes_algorithm, gaussian_kl_div, MLP
from bong.src import bbb, blr, bog, bong, experiment_utils
#from bong.agents import AGENT_NAMES




Matplotlib created a temporary cache directory at /tmp/matplotlib-6ftjutfa because the default path (/teamspace/studios/this_studio/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [36]:
expt = 'iter-blr2'
srcdir = f'~/jobs/paper-{expt}/figs'
dstdir = f'~/bong/bong/results'
for suffix in ['pdf', 'png']:
    src = f'{srcdir}/nlpd_smoothed_jitter_best_lr.{suffix}'
    #src = f'{srcdir}/nlpd_smoothed_jitter.{suffix}'
    dst = f'{dstdir}/{expt}-nlpd.{suffix}'
    cmd = f'cp {src} {dst}'
    print(cmd)
    os.system(cmd)

cp ~/jobs/paper-iter-blr2/figs/nlpd_smoothed_jitter_best_lr.pdf ~/bong/bong/results/iter-blr2-nlpd.pdf
cp ~/jobs/paper-iter-blr2/figs/nlpd_smoothed_jitter_best_lr.png ~/bong/bong/results/iter-blr2-nlpd.png


In [34]:
results_dir = '/teamspace/studios/this_studio/jobs/paper-algo-dlr-medium-n10k-lin-rank'

df = pd.read_csv(f'{results_dir}/jobs.csv')
df.head(n=20)

,jobname,algo,param,lin,dlr_rank,ef,nsample,niter,lr,model_str,key,dataset,data_dim,dgp_type,dgp_str,model_type,ntrain,ntest
0,paper-algo-dlr-medium-n10k-lin-rank-00,bong,diag,1,99,99,99,99,99.000,30_30_1,0,sarcos,0,na,na,mlp,10000,500
1,paper-algo-dlr-medium-n10k-lin-rank-01,bong,dlr,1,1,99,99,99,99.000,30_30_1,0,sarcos,0,na,na,mlp,10000,500
2,paper-algo-dlr-medium-n10k-lin-rank-02,bong,dlr,1,10,99,99,99,99.000,30_30_1,0,sarcos,0,na,na,mlp,10000,500
3,paper-algo-dlr-medium-n10k-lin-rank-03,bog,diag,1,99,99,99,99,0.005,30_30_1,0,sarcos,0,na,na,mlp,10000,500
4,paper-algo-dlr-medium-n10k-lin-rank-04,bog,diag,1,99,99,99,99,0.010,30_30_1,0,sarcos,0,na,na,mlp,10000,500
5,paper-algo-dlr-medium-n10k-lin-rank-05,bog,diag,1,99,99,99,99,0.050,30_30_1,0,sarcos,0,na,na,mlp,10000,500
6,paper-algo-dlr-medium-n10k-lin-rank-06,bog,diag,1,99,99,99,99,0.100,30_30_1,0,sarcos,0,na,na,mlp,10000,500
7,paper-algo-dlr-medium-n10k-lin-rank-07,bog,diag,1,99,99,99,99,0.500,30_30_1,0,sarcos,0,na,na,mlp,10000,500
8,paper-algo-dlr-medium-n10k-lin-rank-08,bog,dlr,1,1,99,99,99,0.005,30_30_1,0,sarcos,0,na,na,mlp,10000,500
9,paper-algo-dlr-medium-n10k-lin-rank-09,bog,dlr,1,10,99,99,99,0.005,30_30_1,0,sarcos,0,na,na,mlp,10000,500


In [18]:

results_dir = '/teamspace/studios/this_studio/jobs/sarcos-mlp2'

df = pd.read_csv(f'{results_dir}/jobs.csv')
#df = pd.read_csv(f'{results_dir}/summary.csv')
df.head(n=20)

,jobname,algo,param,lin,dlr_rank,ef,nsample,niter,lr,model_str,dataset,data_dim,dgp_type,dgp_str,model_type,ntrain
0,sarcos-mlp2-00,bong,dlr,0,10,1,100,99,99.0000,10_1,sarcos,0,na,na,mlp,1000
1,sarcos-mlp2-01,bong,dlr,1,10,99,99,99,99.0000,10_1,sarcos,0,na,na,mlp,1000
2,sarcos-mlp2-02,bog,dlr,0,10,1,100,99,0.0001,10_1,sarcos,0,na,na,mlp,1000
3,sarcos-mlp2-03,bog,dlr,0,10,1,100,99,0.0010,10_1,sarcos,0,na,na,mlp,1000
4,sarcos-mlp2-04,bog,dlr,0,10,1,100,99,0.0100,10_1,sarcos,0,na,na,mlp,1000
5,sarcos-mlp2-05,bog,dlr,1,10,99,99,99,0.0001,10_1,sarcos,0,na,na,mlp,1000
6,sarcos-mlp2-06,bog,dlr,1,10,99,99,99,0.0010,10_1,sarcos,0,na,na,mlp,1000
7,sarcos-mlp2-07,bog,dlr,1,10,99,99,99,0.0100,10_1,sarcos,0,na,na,mlp,1000
8,sarcos-mlp2-08,blr,dlr,0,10,1,100,1,0.0001,10_1,sarcos,0,na,na,mlp,1000
9,sarcos-mlp2-09,blr,dlr,0,10,1,100,10,0.0001,10_1,sarcos,0,na,na,mlp,1000


In [17]:

results_dir = '/teamspace/studios/this_studio/jobs/sarcos-mlp18'

df = pd.read_csv(f'{results_dir}/jobs.csv')
#df = pd.read_csv(f'{results_dir}/summary.csv')

df.head(n=20)




,jobname,algo,param,lin,dlr_rank,ef,nsample,niter,lr,model_str,key,dataset,data_dim,dgp_type,dgp_str,model_type,ntrain,ntest
0,sarcos-mlp18-00,bong,dlr,0,1,1,100,99,99.000,10_1,0,sarcos,0,na,na,mlp,2000,500
1,sarcos-mlp18-01,bong,dlr,0,1,1,100,99,99.000,20_20_1,0,sarcos,0,na,na,mlp,2000,500
2,sarcos-mlp18-02,bong,dlr,1,1,99,99,99,99.000,10_1,0,sarcos,0,na,na,mlp,2000,500
3,sarcos-mlp18-03,bong,dlr,1,1,99,99,99,99.000,20_20_1,0,sarcos,0,na,na,mlp,2000,500
4,sarcos-mlp18-04,bog,dlr,0,1,1,100,99,0.005,10_1,0,sarcos,0,na,na,mlp,2000,500
5,sarcos-mlp18-05,bog,dlr,0,1,1,100,99,0.005,20_20_1,0,sarcos,0,na,na,mlp,2000,500
6,sarcos-mlp18-06,bog,dlr,0,1,1,100,99,0.010,10_1,0,sarcos,0,na,na,mlp,2000,500
7,sarcos-mlp18-07,bog,dlr,0,1,1,100,99,0.010,20_20_1,0,sarcos,0,na,na,mlp,2000,500
8,sarcos-mlp18-08,bog,dlr,0,1,1,100,99,0.050,10_1,0,sarcos,0,na,na,mlp,2000,500
9,sarcos-mlp18-09,bog,dlr,0,1,1,100,99,0.050,20_20_1,0,sarcos,0,na,na,mlp,2000,500


In [ ]:
#condition = (df['minscore'] != df[args.metric])
condition = (df['lin']==1) | (df['ef']==0)
indices_to_drop = df[condition].index
df_filtered = df.drop(indices_to_drop)

df_filtered.head(n=20)

In [1]:
D = 22 # since add one for sarcos
# For FC
mlp = [(D+1)*n + (n+1)*1 for n in [10, 20, 30, 40]] # 1 layer
print(mlp)

# For DLR
mlp = [(D+1)*n + (n+1)*n  + (n+1)*1 for n in [10, 20, 30]] # 2 layer MLP
print(mlp)

mlp = [(D+1)*n + (n+1)*n + (n+1)*n + (n+1)*1 for n in [10, 20, 30]] # 3 layer MLP
print(mlp)


mlp=  (784+1)*50 + (50+1)*50  + (50+1)*10
print(mlp)



[241, 481, 721, 961]
[351, 901, 1651]
[461, 1321, 2581]
42310


In [8]:
import os
import argparse
import os
import itertools
import pandas as pd
from pathlib import Path
import os
import datetime

def list_subdirectories(directory):
    return [name for name in os.listdir(directory)
            if os.path.isdir(os.path.join(directory, name))]

src = '/teamspace/jobs'
dst = '/teamspace/studios/this_studio/jobs'
dirs = list_subdirectories(src)
for old_name in dirs:
    parts = old_name.split('-')
    num = parts[2]
    new_name = f'expt_timing2-{num}'
    dst_path = Path(dst, new_name, 'work')
    print(f'\n Creating {str(dst_path)}')
    dst_path.mkdir(parents=True, exist_ok=True)

    #cmd = f'mv {directory}/{old_name} {directory}/{new_name}'
    cmd = f'sudo cp -r {src}/{old_name}/work/*.* {dst}/{new_name}/work'
    print(cmd)
    os.system(cmd)


 Creating /teamspace/studios/this_studio/jobs/expt_timing2-13/work
sudo cp -r /teamspace/jobs/expt-timing2-13/work/*.* /teamspace/studios/this_studio/jobs/expt_timing2-13/work

 Creating /teamspace/studios/this_studio/jobs/expt_timing2-10/work
sudo cp -r /teamspace/jobs/expt-timing2-10/work/*.* /teamspace/studios/this_studio/jobs/expt_timing2-10/work

 Creating /teamspace/studios/this_studio/jobs/expt_timing2-02/work
sudo cp -r /teamspace/jobs/expt-timing2-02/work/*.* /teamspace/studios/this_studio/jobs/expt_timing2-02/work

 Creating /teamspace/studios/this_studio/jobs/expt_timing2-00/work
sudo cp -r /teamspace/jobs/expt-timing2-00/work/*.* /teamspace/studios/this_studio/jobs/expt_timing2-00/work

 Creating /teamspace/studios/this_studio/jobs/expt_timing2-15/work
sudo cp -r /teamspace/jobs/expt-timing2-15/work/*.* /teamspace/studios/this_studio/jobs/expt_timing2-15/work

 Creating /teamspace/studios/this_studio/jobs/expt_timing2-41/work
sudo cp -r /teamspace/jobs/expt-timing2-41/work

In [3]:
rank, nsamples, niter, lr = 10, 20, 30, 0.01
algo = 'bog'
s = make_full_name(algo, 'dlr', rank, 1, 0, nsamples, niter, lr)
print(s)
parts = parse_full_name(s)
print(parts)

NameError: name 'keys' is not defined